# Data Cleaning

* Now the data mining is complete
* Here we focus on preparing the data for analysis
* Due to a low count of rows '3588', we will take best effort to not loose/delete any data

In [1]:
import pandas as pd
import seaborn as sns
from warnings import filterwarnings
filterwarnings("ignore")

# Reading csv

In [2]:
df = pd.read_csv('BA_review_forage.csv')
df.head()

,Unnamed: 0,reviews,stars,date,country
0,0,✅ Trip Verified | My flight on on 12 May 2023...,1,29th June 2023,United Arab Emirates
1,1,Not Verified | Cairo is a 5 hour flight and B...,2,29th June 2023,United Kingdom
2,2,✅ Trip Verified | After travelling London to ...,1,27th June 2023,United Kingdom
3,3,✅ Trip Verified | My luggage was mis-tagged i...,1,27th June 2023,United States
4,4,✅ Trip Verified | The airline lost my luggage...,1,25th June 2023,United States


In [3]:
df = df.drop(['Unnamed: 0'], axis=1)

In [4]:
df.head()

,reviews,stars,date,country
0,✅ Trip Verified | My flight on on 12 May 2023...,1,29th June 2023,United Arab Emirates
1,Not Verified | Cairo is a 5 hour flight and B...,2,29th June 2023,United Kingdom
2,✅ Trip Verified | After travelling London to ...,1,27th June 2023,United Kingdom
3,✅ Trip Verified | My luggage was mis-tagged i...,1,27th June 2023,United States
4,✅ Trip Verified | The airline lost my luggage...,1,25th June 2023,United States


In [5]:
df.shape

(3588, 4)

In [6]:
df['reviews'][0]

'✅ Trip Verified |  My flight on on 12 May 2023 got delayed an hour and 25 minutes which resulted in the cancellation of our connection flight on 12 May 2023 as the arrival of the first flight was only 15 before the departure of the second connecting flight. We contacted the BA call center multiple times to make changes to the flight in order to still arrive on the 12th of May to KEF airport through different flights. The team tried to find a solution but none made sense, not even on the next day the 13th of May 2023. We have made full booking of accommodation, car rental, activities etc for the vacation in Iceland. The cancellation of of these booking is not refundable. The call center suggested that we cancel the flight for a full refund, then claim compensation for the delay and difference in airfare cost and other non refundable bookings due to the delay and cancellation of our flight. British Airways responded to the claim after a month with no compensation at all, even though we 

# Normalising the text

In [7]:
from bs4 import BeautifulSoup
import spacy
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
import unicodedata

In [8]:
nlp = spacy.load('en_core_web_sm')

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

Remove HTML tags

In [11]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

Remove accented characters

In [12]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

Remove special characters

In [13]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

Lemmatization

In [14]:
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

Stemming

In [15]:
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

Removing Stopwords

In [17]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

# Text Normalization

In [18]:
def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    
    for doc in corpus:
        
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
            
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
            
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
            
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
            
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits) 
            
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

In [19]:
df['corpus'] = normalize_corpus(df['reviews'])

In [20]:
df.head()

,reviews,stars,date,country,corpus
0,✅ Trip Verified | My flight on on 12 May 2023...,1,29th June 2023,United Arab Emirates,trip verify flight may got delay hour minute r...
1,Not Verified | Cairo is a 5 hour flight and B...,2,29th June 2023,United Kingdom,not verify cairo hour flight ba consider short...
2,✅ Trip Verified | After travelling London to ...,1,27th June 2023,United Kingdom,trip verify travel london madrid british airwa...
3,✅ Trip Verified | My luggage was mis-tagged i...,1,27th June 2023,United States,trip verify luggage mis tag dallas way cairo v...
4,✅ Trip Verified | The airline lost my luggage...,1,25th June 2023,United States,trip verify airline lose luggage absolutely aw...


# Cleaning

Date Column

In [21]:
# Changing the date dtype to useable format

df['date'] = pd.to_datetime(df['date'])

In [22]:
df.head()

,reviews,stars,date,country,corpus
0,✅ Trip Verified | My flight on on 12 May 2023...,1,2023-06-29,United Arab Emirates,trip verify flight may got delay hour minute r...
1,Not Verified | Cairo is a 5 hour flight and B...,2,2023-06-29,United Kingdom,not verify cairo hour flight ba consider short...
2,✅ Trip Verified | After travelling London to ...,1,2023-06-27,United Kingdom,trip verify travel london madrid british airwa...
3,✅ Trip Verified | My luggage was mis-tagged i...,1,2023-06-27,United States,trip verify luggage mis tag dallas way cairo v...
4,✅ Trip Verified | The airline lost my luggage...,1,2023-06-25,United States,trip verify airline lose luggage absolutely aw...


Star column

In [23]:
df['stars'].unique()

array(['1', '2', '4', '7', '3', '9', '10', '5', '8',
       '\n\t\t\t\t\t\t\t\t\t\t\t\t\t5', '6', 'None'], dtype=object)

In [24]:
# remove the \n\t.. from the ratings

df['stars'] = df['stars'].str.strip("\n\t\t\t\t\t\t\t\t\t\t\t\t\t")

In [25]:
df['stars'].unique()

array(['1', '2', '4', '7', '3', '9', '10', '5', '8', '6', 'None'],
      dtype=object)

In [26]:
round(df['stars'].value_counts(normalize=True),2)

1       0.22
2       0.11
3       0.11
8       0.10
10      0.09
7       0.09
9       0.09
5       0.07
4       0.07
6       0.05
None    0.00
Name: stars, dtype: float64

Finding Columns with no star values

In [27]:
df[df['stars'] =='None']

,reviews,stars,date,country,corpus
3040,Los Angeles (LAX) to London (LHR) in Club Worl...,None,2015-01-26,United Kingdom,los angeles lax london lhr club world january ...
3170,Just completed a return trip to Hong Kong on t...,None,2014-11-20,United Kingdom,complete return trip hong kong magnificent air...
3190,LHR-JNB 15/10. I cannot understand why BA have...,None,2014-11-12,France,lhr jnb I not understand ba choose seating arr...
3428,Very disappointed with my flight back from Mun...,None,2014-07-15,United Kingdom,disappointed flight back munich although hour ...
3463,Flight BA 202 Boston to Heathrow. Aircraft B74...,None,2014-06-23,United Kingdom,flight ba boston heathrow aircraft b version s...


In [28]:
df['reviews'][3040] #8

"Los Angeles (LAX) to London (LHR) in Club World. January 2015. Have not flown out of the Tom Bradley International Terminal at LAX for years but it's now much improved. Check in and bag drop a breeze as was security and in the oneworld lounge in minutes. Good choice of food and drinks and plenty of space. Short-ish walk to the gate good welcome on board and drinks served quickly. Couldn't fault any aspect of the experience and we took off and landed on time and managed a good five hours sleep out of the ten hours duration. Love the Arrivals Lounge at LHR. A civilising shower and 2nd breakfast sets you up for the day."

In [29]:
df['reviews'][3170] #4

'Just completed a return trip to Hong Kong on the A380 which is a magnificent aircraft. The crew food and inflight entertainment were good. We travelled economy and paid an extra £140 to reserve 2 seats on the upper deck. We found the seats uncomfortable and when the person in front reclined their seat you were trapped. Leg room was restricted by the video box. We were sat 2 rows from the toilet which were nearly constantly in use and the noise from the toilet flush was an irritating. The inflight entertainment just kept me sane as I found it impossible to be comfortable.'

In [30]:
df['reviews'][3190] #4

'LHR-JNB 15/10. I cannot understand why BA have chosen the seating arrangement in Club World on the new A380 with one seat facing forward and the seat next to you facing backwards so as to be facing your neighbour. Food reasonable and service OK but not overly attentive. Return flight in Premium Economy seats not as comfortable as in the 767.'

In [31]:
df['reviews'][3428] #5

'Very disappointed with my flight back from Munich although it\'s only 2 hours all that was offered was a small basket consisting of a packet of nuts or shortbread. The flight over to Munich was considerably better being offered chicken curry wrap etc. We were told when booking this return in Munich that there was only one seat left however once in the aircraft there were still seats left including one next to me! However it\'s really the snack offered that\'s my main "bone of contention" as we chose British Airway in preference to the cheap budget airlines but we now find there\'s little difference in the on flight catering.'

In [32]:
df['reviews'][3463] #1

'Flight BA 202 Boston to Heathrow. Aircraft B747 version 1. We were seated in PE. The cabin was quiet and not full so we had plenty of room. The cabin crew were very attentive and spoke to all the passengers in our cabin. Service after take off was very quick and professional the food was very good and not hurried giving us 3.5 hours to sleep. The seats were very comfortable and the extra space was appreciated. My only complaint is my TV monitor was faulty and had a black line down the screen. It was not an issue as I was planning to sleep rather than watch a movie. The cabin was quieter than economy as it was at the front of the plane and I had a good 3 hours sleep. The Flight attendants were offering drinks throughout the flight which was good. The breakfast was a croissant in a box which I found a bit of a disappointment. Overall I would fly BA Premium Economy again.'

Manually assigning the star rating based on the reviews, carefully considering the wordings used to make atmost sense

In [33]:
df['stars'][3040] = 8
df['stars'][3170] = 4
df['stars'][3190] = 4
df['stars'][3428] = 5
df['stars'][3463] = 7

In [34]:
df['stars'].value_counts()

1     804
2     410
3     396
8     353
10    318
7     307
9     305
5     265
4     240
6     185
4       2
8       1
5       1
7       1
Name: stars, dtype: int64

Country column

In [35]:
df.isna().sum()

reviews    0
stars      0
date       0
country    2
corpus     0
dtype: int64

In [36]:
df[df.isna().any(axis=1)]

,reviews,stars,date,country,corpus
2946,I travelled from London to Jo'burg and back on...,5,2015-04-08,NaN,I travel london joburg back airbus mixed exper...
3250,St Lucia to London round trip. Full flight bot...,8,2014-10-20,NaN,st lucia london round trip full flight way qui...


In [37]:
df['country'].value_counts()

United Kingdom           2269
United States             394
Australia                 154
Canada                    111
Germany                    59
                         ... 
Saint Kitts and Nevis       1
Panama                      1
Chile                       1
Jordan                      1
Oman                        1
Name: country, Length: 70, dtype: int64

In [38]:
df['reviews'][2946]

"I travelled from London to Jo'burg and back on the Airbus 380 and had mixed experiences. On the way out I was in Premium Economy and was pleasantly surprised by the spacious seat and window locker and slept well. The return in Business was a different matter. I was truly shocked by how many seats had been crammed in the uncomfortable narrow seat the lack of privacy and having to climb over passenger's feet to vacate my seat. All this at a premium price. Shocking. The cabin crew were faultless and were only too aware of passengers' views on the cabin! Never again."

In [39]:
df['reviews'][3250]

'St Lucia to London round trip. Full flight both ways quite unhappy staff on return leg tiny seatback screens and poor selection of entertainment meals satisfactory (taste was ok and enough to fill me up both legs on time but was quite expensive for the service I got. Would fly them again if they are the cheaper option.'

Going through the reviews, we identified the countries, so lets replace them in the null values

In [40]:
df['country'].unique()

array(['United Arab Emirates', 'United Kingdom', 'United States',
       'Romania', 'Australia', 'Greece', 'Canada', 'Cyprus', 'Italy',
       'Spain', 'Chile', 'Sweden', 'Ireland', 'Hong Kong', 'Netherlands',
       'Austria', 'France', 'India', 'Belgium', 'New Zealand',
       'Czech Republic', 'Malaysia', 'Singapore', 'South Africa', 'Ghana',
       'Germany', 'Switzerland', 'Bermuda', 'Botswana', 'Brazil',
       'Panama', 'Nigeria', 'Russian Federation', 'Philippines',
       'Bulgaria', 'Poland', 'Thailand', 'Argentina', 'Mexico', 'Denmark',
       'Saint Kitts and Nevis', 'Vietnam', 'Norway', 'Jordan', 'Japan',
       'Taiwan', 'China', 'Slovakia', 'Kuwait', 'Israel', 'Qatar',
       'South Korea', 'Saudi Arabia', 'Hungary', 'Portugal',
       'Cayman Islands', 'Costa Rica', 'Egypt', 'Iceland', 'Laos',
       'Turkey', 'Indonesia', 'Bahrain', 'Dominican Republic',
       'Luxembourg', 'Finland', 'Ukraine', nan, 'Trinidad & Tobago',
       'Barbados', 'Oman'], dtype=object)

In [41]:
df['country'][2946] = 'United Kingdom'
df['country'][3250] = 'United Kingdom'

In [42]:
df.isna().sum()

reviews    0
stars      0
date       0
country    0
corpus     0
dtype: int64

In [43]:
df['country'].value_counts()

United Kingdom           2271
United States             394
Australia                 154
Canada                    111
Germany                    59
                         ... 
Saint Kitts and Nevis       1
Panama                      1
Chile                       1
Jordan                      1
Oman                        1
Name: country, Length: 70, dtype: int64

In [44]:
df.shape

(3588, 5)

The data is clean, we can now proceed for Analysis

In [45]:
import os

cwd = os.getcwd()
df.to_csv(cwd+ "/BA_reviews_forage_clean.csv")